In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import os
from ds_capability import FeatureSelect, FeatureEngineer, AutoML, Controller

## Synthesize

In [4]:
syn_fe = FeatureEngineer.from_memory()
syn_ds = FeatureSelect.from_memory()

syn_fe.add_connector_uri('sample', 'https://raw.githubusercontent.com/project-hadron/hadron-asset-bank/master/datasets/toy_sample/churn.csv')

syn_fe.set_persist_uri('./hadron/data/synthetic_churn.parquet')

In [5]:
tbl = syn_fe.tools.get_analysis_group(size=10_000, other='sample', group_by='Exited')
tbl = syn_fe.tools.get_number(16_000_000, 19_999_999, at_most=1, size=10_000,
                              relative_freq=[99, 20, 5, 1],
                              to_header='CustomerId', canonical=tbl)
tbl.shape

(10000, 14)

In [6]:
tbl = syn_ds.tools.auto_drop_columns(tbl, headers=['Exited'])
tbl.shape

(10000, 13)

In [7]:
syn_fe.save_persist_canonical(tbl)

In [8]:
result = syn_fe.load_persist_canonical()
result.shape

(10000, 13)

## Remote Run

In [9]:
os.environ['HADRON_CHURN_SOURCE_PATH'] = './hadron/data/synthetic_churn.parquet'
os.environ['HADRON_CHURN_PERSIST_PATH'] = './hadron/data/hadron_docs_churn_predict.parquet'
os.environ['HADRON_PM_REPO'] = 'https://raw.githubusercontent.com/project-hadron/hadron-asset-bank/master/contracts/pyarrow/docs/use_case_two/'

In [10]:
ctrl = Controller.from_env()

In [11]:
ctrl.run_controller()

## View Remote

In [12]:
AutoML.from_env('churn').load_persist_canonical()

pyarrow.Table
CustomerId: int64
predict: int64
----
CustomerId: [[16098142,16106220,16486515,16036382,16788702,...,16691005,16600877,16912351,16056608,16748186]]
predict: [[0,0,0,0,0,...,0,0,0,1,0]]